# Exercise 6: Building a thermochronology model (20 points)

This week we will combine some of the codes created earlier in the course to create a numerical model for predicting thermochronometer ages and comparing them to measured ages.

### Tips for completing this exercise

- Use **exactly** the same variable names as in the instructions because your answers will be automatically graded, and the tests that grade your answers rely on following the same formatting or variable naming as in the instructions.
- **Please do not**:

    - **Change the file names**. Do all of your editing in the provided `Exercise-6-problems-1-2.ipynb` file (this file).
    - **Copy/paste cells in this notebook**. We use an automated grading system that will fail if there are copies of code cells.
    - **Change the existing cell types**. You can add cells, but changing the cell types for existing cells (from code to markdown, for example) will also cause the automated grader to fail.

## AI tool usage agreement

**Enter your name in the cell below** to confirm that you have followed the [course guidelines on the use of AI tools](https://introqg-site.readthedocs.io/en/latest/general-info/ai-tools.html) and understand that misuse of AI tools is considered cheating.

YOUR ANSWER HERE

## Problem 1: Filling in the missing pieces (14 points)

The overall goal of this exercise is to create a model for calculating thermochronometer ages as a function of the advection velocity in a 1D thermal model in order to try to find the long-term exhumation rate that best fits a set of measured ages. We are going to assume that erosion balances any potential uplift of the Earth's surface such that the advection velocity in the thermal model will be equivalent to the long-term exhumation rate. An overview of how we can go about calculating thermochronometer ages from a thermal model is given below. Items in italics will be handled in Exercise 7.

### Predicting (or calculating) thermochronometer ages from a thermal model

1. Create a temperature history for a rock that has cooled during exhumation to the Earth's surface
    1. Define a simulation duration and time-depth history
    2. Calculate a temperature history for the rock
        - We already have the model for calculating temperatures over time, so we just need to store the temperatures for each depth and time in the time-depth history (this is fairly easy)
2. Use the temperature history to calculate the effective closure temperatures for different thermochronometers using Dodson's method
3. Use the closure temperatures to calculate thermochronometer ages from the temperature history
4. *Read in the measured age data*
5. *Calculate a goodness of fit for each thermochronometer system*
6. Calculate temperatures for the crust at the start and end of thermal model calculation (for plotting)
7. Plot the results

Overall, this might look scary, but I will again assure you we already have the pieces to do most of the difficult stuff in the list of tasks above.

### Scores for this problem

**Your score on this problem will be based on following criteria**:

- Creating some new functions for calculating thermochronometer ages
- Testing your functions
- Creating a numerical model to calculate thermochronometer ages using your functions
- Including comments that explain what most lines in the code do
- Uploading your notebook to your GitHub repository for this week's exercise

### Part 0: Copying and testing your script file from Exercise 5 (0 points)

The first task in this problem is to copy your `introqg_functions.py` script file from Exercise 5 to the directory containing this notebook and then run the cell below to ensure it has been copied and is functioning as expected. Note: We will only check some of the functions in this file using the tests below, not all of them.

- Copy your `introqg_functions.py` script file from Exercise 5 to the directory containing this notebook
- Run the tests below

In [ ]:
# The test below should work

from nose.tools import assert_equal
from introqg_functions import dodson, transient_temp

# Calculate some closure temperatures using Dodson's method
tc1 = dodson(
    cooling_rate=-4.0,
    activation_energy=138.0e3,
    diffusivity_inf=5.0e-3,
    grain_radius=100.0,
    geometry_factor=25
)
tc2 = dodson(
    cooling_rate=-8.0,
    activation_energy=168.0e3,
    diffusivity_inf=4.6e-5,
    grain_radius=100.0,
    geometry_factor=25
)
tc3 = dodson(
    cooling_rate=-14.0,
    activation_energy=264.0e3,
    diffusivity_inf=2.3e-4,
    grain_radius=500.0,
    geometry_factor=27
)

# Print calculated closure temperature
print(f"Calculated apatite (U-Th)/He closure temperature: {tc1:7.3f} °C")
print(f"Calculated zircon (U-Th)/He closure temperature:  {tc2:7.3f} °C")
print(f"Calculated muscovite Ar/Ar closure temperature:   {tc3:7.3f} °C")
print("")

# Check that the closure temperature values are correct
assert_equal(round(tc1, 3), 72.566)
assert_equal(round(tc2, 3), 196.282)
assert_equal(round(tc3, 3), 492.379)

# Test the transient temperature calculation
test_temp1 = transient_temp(
    initial_gradient=18.0,
    diffusivity=32.0,
    velocity=0.7,
    depths=18.0,
    time=35.0)

test_temp2 = transient_temp(
    initial_gradient=13.0,
    diffusivity=32.0,
    velocity=1.2,
    depths=13.0,
    time=25.0)

# Print calculated closure temperature
print(f"Calculated transient temperature 1: {test_temp1:7.3f} °C")
print(f"Calculated transient temperature 2: {test_temp2:7.3f} °C")

# Check that the closure temperature values are correct
assert_equal(round(test_temp1, 3), 580.012)
assert_equal(round(test_temp2, 3), 389.873)

# Print message if it is safe to continue
print("\nAll tests pass! You are ready to proceed with this exercise.")

### Part 1: Creating a thermal history function (3 points)

Here, we will create a function to generate a thermal history for a rock as it travels through the crust while being advected toward the surface. We will later use this recorded temperature history to predict thermochronometer ages.

The key to understanding what is done here is to understand that we will be simulating the position of a parcel of rock at depth in the earth, and at each time step the position of the rock parcel will be moved upward according to the length of the time interval between steps in the time history multiplied by the advection velocity.
In mathematical terms, this relationship is

\begin{equation}
  \large
  z(t) = z(t-1) - v_{z} dt
\end{equation}
*Equation 1. Equation for calculating rock particle depth as a function of time.*

Thus, we will track a parcel of rock from some depth in the model at the start of the temperature calculation to its final location at the surface when the simulation is complete at 0 Ma.
At each depth, the temperature of the parcel of rock will be stored, which will allow the cooling in the temperature history of the rock parcel to be used to predict different thermochronometer ages.

**Creating the function**

Below you should create a new function in your `introqg_functions.py` script file called `calculate_temp_history()`. Your function should:

- Start by defining a NumPy array of zeros in a variable called `temp_history`
    - You can use the NumPy function `np.zeros()` to do this. Check out the [np.zeros documentation](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) to see how to use this function.
    - The array of zeros should have the same number of values as the time history
- Use a for loop to iterate over all of the values in your time-depth history array and calculate a temperature for each combination of time and depth
    - I recommend you use the variable `i` with the `range(len())` function combination for the loop, with the time history as the array whose length you use to define the range of values of `i`
        - This way you can store a temperature value for each value of `i` using the corresponding value of the time and depth histories
    - You can calculate the temperature for a single time and depth value from the time and depth history arrays using the `transient_temp` function from your `introqg_functions.py` script file
- At the end of the loop, return the `temp_history` array, which should contain one temperature for each depth-time pair

In the function, you will pass in the following values, and you should use the same name for the parameters in your `calculate_temp_history()` function:

- `time_history` (Array of time values for thermal history. **Note**: Time here is thermal model time, not time in millions of years ago; units: Myr)
- `depth_history` (Array of depth values for thermal history; units: km)
- `velocity` (Advection velocity; units: km / Myr)
- `initial_gradient` (Temperature gradient at start of thermal model calculation; units: deg. C / km)
- `diffusivity` (Thermal diffisivity; units: km^2 / Myr)

**NOTE**: There is no need to convert units from the units listed above in your function. Distances in kilometers and time in millions of years (Myr) will work without conversion.

**What to do for this part:**

- Create a function called `calculate_temp_history()` in your `introqg_functions.py` script file
    - Your function parameters should be `time_history`, `depth_history`, `velocity`, `gradient`, and `diffusivity` as described above

In [ ]:
# Import your calculate_temp_history function below

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# The test below should work

# Import libraries we need
import numpy as np
from nose.tools import assert_equal
from introqg_functions import calculate_temp_history

# Define advection velocity, total time, and number of points for time and depth histories
vz_test1 = 0.5
total_time = 50.0
num_pts = 401

# Create time history
time_hist_test1 = np.linspace(0.0, total_time, num_pts)

# Create depth history
max_depth = vz_test1 * total_time
depth_hist_test1 = np.linspace(max_depth, 0.0, num_pts)

# Create temperature history using calculate_temp_history function
temp_hist_test1 = calculate_temp_history(
    time_history=time_hist_test1,
    depth_history=depth_hist_test1,
    velocity=vz_test1,
    initial_gradient=10.0,
    diffusivity=32.0
)

# Print test temperature value
test_index1 = 42
print(f"Temperature after {time_hist_test1[test_index1]} Myr at {depth_hist_test1[test_index1]} km depth: {temp_hist_test1[test_index1]:.3f} °C")

# Check that the temperature value is correct
assert_equal(round(temp_hist_test1[test_index1], 3), 248.005)

In [ ]:
# The test below should work

# Print test temperature value
test_index2 = 27
print(f"Temperature after {time_hist_test1[test_index2]} Myr at {depth_hist_test1[test_index2]} km depth: {temp_hist_test1[test_index2]:.3f} °C")

# Check that the temperature value is correct
assert_equal(round(temp_hist_test1[test_index2], 3), 249.488)

In [ ]:
# The test below should work

# Define advection velocity, total time, and number of points for time and depth histories
vz_test3 = 0.85

# Create time history
time_hist_test3 = np.linspace(0.0, total_time, num_pts)

# Create depth history
max_depth = vz_test3 * total_time
depth_hist_test3 = np.linspace(max_depth, 0.0, num_pts)

# Create temperature history using calculate_temp_history function
temp_hist_test3 = calculate_temp_history(
    time_history=time_hist_test3,
    depth_history=depth_hist_test3,
    velocity=vz_test3,
    initial_gradient=15.0,
    diffusivity=32.0
)

# Print test temperature value
test_index3 = 365
print(f"Temperature after {time_hist_test3[test_index3]} Myr at {depth_hist_test3[test_index3]} km depth: {temp_hist_test3[test_index3]:.3f} °C")

# Check that the temperature value is correct
assert_equal(round(temp_hist_test3[test_index3], 3), 146.497)

### Part 2: A function for calculating closure temperatures (6 points)

The next function we need to create is one for calculating the effective closure temperature from the thermal history that is produced by the `calculate_temp_history()` function. You might wonder why we need this when we already have our `dodson()` function from Exercise 3. Well, the reason we need this additional function is that we now have a thermal history with a cooling rate that might vary over time, and we need to ensure we are using the cooling rate that applies at the time when the rock parcel is cooling through the effective closure temperature. We can create our function as follows:

**Creating the function**

Below you should create a new function in your `introqg_functions.py` script file called `calculate_closure_temps()`. In the function, you will pass in the following values, and you should use the same name for the parameters in your `calculate_closure_temps()` function:

- `time_history` (Array of time values. **Note**: Time here is thermal model time, not time in millions of years ago; units: Myr)
- `temp_history` (Array of temperature values ; units: °C)
- `calc_ahe` (Calculate apatite (U-Th)/He closure temperature?; `True` or `False`)
- `calc_zhe` (Calculate zircon (U-Th)/He closure temperature?; `True` or `False`)
- `calc_mar` (Calculate muscovite Ar/Ar closure temperature?; `True` or `False`)

Your function should:

- Start by setting the effective closure temperature to `None` for the thermochronometers we will consider
    - We will use the apatite (U-Th)/He, zircon (U-Th)/He, and muscovite Ar/Ar thermochronometers
    - In your function, I suggest using variable names such as `final_ahe_tc` for storing the closure temperature for the apatite (U-Th)/He thermochronometer, and similar names for other thermochronometers
- Next, you can calculate an array of cooling rates for the thermal history you created using the `calculate_temp_history()` function
    - First, calculate the time increment between values in the time history as the absolute value of the difference between the first two times in the time history array. This will ensure the time increment is a positive number.
    - After that, you can create an array of cooling rates using the `np.gradient()` function. In our case, we will use the temperatures as the source array along with the time increment in order to calculate the changes in temperature with time (i.e., the cooling rates). Check out the [np.gradient documentation](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html) to see how to use this function.
- Finally, you will again use a `for` loop to iterate over all of the values in the time history and calculate the effective closure temperatures
    - You should again use a `for` loop along with the `range(len())` construct to iterate over the values in the time history array
    - Once inside the loop, you should ensure that the cooling rate is at least 0.1 °C/Myr in each iteration by taking the minimum of the cooling rate at index `i` in your cooling rates array and the value `-0.1`. You can use the `min()` function for this. Because the cooling rates will be negative, the minimum function will find the larger negative value. You can assign the output from the `min()` function to a variable such as `cooling_rate_min` and use that for your closure temperature calculations.
    - Now that you have a suitable cooling rate in the loop, you can use your `dodson()` function to calculate closure temperatures
        - Only calculate a closure temperature if the corresponding boolean variable (`calc_ahe`, for example) is `True`
        - If so, call the `dodson()` function using the adjusted cooling rate (i.e., `cooling_rate_min` for the cooling rate) and with the correct parameters for the given thermochronometer (see Table 1 below). You can store the output in an appropriate variable, such as `ahe_tc`.
        - After calculating the effective closure temperature, you should check to see whether that closure temperature is lower than the current temperature in the temperature history array for this iteration
            - If yes, then store the newly calculated closure temperature with a variable name such as `final_ahe_tc`
            - This will ensure that the closure temperature that is stored will only be updated while the temperature in the temperature history is above the effective closure temperature
        - You can repeat the preceding steps for the other two remaining thermochronometers being sure to calculate the effective closure temperature only if the passed `calc_zhe` or `calc_mar` parameter variable is `True` and storing the "final" closure temperature only when the temperature in the temperature history is above the calculated closure temperature
- After calculating the closure temperatures, you can return all three "final values" (e.g., `final_ahe_tc`) in a single `return` statement

The parameters to use for the closure temperature calculations are below.

| Thermochronometer | $E_{\mathrm{a}}$      | $D_{0}$              | $a$   | $A$ |
|-------------------|:---------------------:|:--------------------:|:-----:|:---:|
| Apatite (U-Th)/He | $138.0 \times 10^{3}$ | $5.0 \times 10^{-3}$ | 100.0 | 25  |
| Zircon (U-Th)/He  | $168.0 \times 10^{3}$ | $4.6 \times 10^{-5}$ | 100.0 | 25  |
| Muscovite Ar/Ar   | $264.0 \times 10^{3}$ | $2.3 \times 10^{-4}$ | 500.0 | 27  |

*Table 1. Parameters for effective closure temperature calculations. $E_{\mathrm{a}}$ is the activation energy, $D_{0}$ is the diffusivity at infinite temperature, $a$ is the grain radius, and $A$ is the geometry factor.*

**What to do for this part:**

- Create a function called `calculate_closure_temps()` in your `introqg_functions.py` script file
    - Your function parameters should be `time_history`, `temp_history`, `calc_ahe`, `calc_zhe`, and `calc_mar` as described above

In [ ]:
# Import your calculate_closure_temps function below

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# The test below should work

# Import libraries we need
from nose.tools import assert_equal
from introqg_functions import calculate_closure_temps

# Create time history
time_hist_test1 = np.array([0.0, 50.0])

# Create temperature history
temp_hist_test1 = np.array([500.0, 0.0])

# Calculate closure temperatures
ahe_tc1, zhe_tc1, mar_tc1 = calculate_closure_temps(
    time_history=time_hist_test1,
    temp_history=temp_hist_test1,
    calc_ahe=True,
    calc_zhe=False,
    calc_mar=True
)

# Print closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc1:.1f} °C. Expected value: 79.0 °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc1}. Expected value: None.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc1:.1f} °C. Expected value: 486.5 °C.")

# Check that the AHe closure temperature is correct
assert_equal(round(ahe_tc1, 3), 79.016)

In [ ]:
# The test below should work

# Create time history
time_hist_test2 = np.array([0.0, 50.0])

# Create temperature history
temp_hist_test2 = np.array([250.0, 0.0])

# Calculate closure temperatures
ahe_tc2, zhe_tc2, mar_tc2 = calculate_closure_temps(
    time_history=time_hist_test2,
    temp_history=temp_hist_test2,
    calc_ahe=True,
    calc_zhe=True,
    calc_mar=True
)

# Print closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc2:.1f} °C. Expected value: 74.1 °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc2:.1f}. Expected value: 191.4 °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc2}. Expected value: None.")

# Check that the ZHe closure temperature is correct
assert_equal(round(zhe_tc2, 3), 191.434)

In [ ]:
# The test below should work

# Create time history
time_hist_test3 = np.array([0.0, 25.0])

# Create temperature history
temp_hist_test3 = np.array([600.0, 0.0])

# Calculate closure temperatures
ahe_tc3, zhe_tc3, mar_tc3 = calculate_closure_temps(
    time_history=time_hist_test3,
    temp_history=temp_hist_test3,
    calc_ahe=True,
    calc_zhe=True,
    calc_mar=True
)

# Print closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc3:.1f} °C. Expected value: 85.4 °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc3:.1f} °C. Expected value: 208.0 °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc3:.1f} °C. Expected value: 502.0 °C.")

# Check that the temperature value is correct
assert_equal(round(mar_tc3, 3), 501.986)

### Part 3: A function for calculating thermochronometer ages (3 points)

Now that we have a closure temperature for our thermochronometers of interest, we can move on to calculating a thermochronometer age using that closure temperature. Compared to the previous function, this will be quite straightforward.

At this point we have a thermal history from the combination of depth-time and temperature values, and a closure temperature. In order to calculate an age, we simply need to determine the time at which we reach the effective closure temperature in the thermal history. We will do this using a linear interpolation of the thermal history, as described below.

**Creating the function**

Below you should create a new function in your `introqg_functions.py` script file called `calculate_age()`. In the function, you will pass in the following values, and you should use the same name for the parameters in your `calculate_age()` function:

- `time_history` (Array of time values. **Note**: Time here is thermal model time, not time in millions of years ago; units: Myr)
- `temp_history` (Array of temperature values ; units: °C)
- `closure_temp` (The closure temperature; units: °C)

Your function should:

- Use a linear interpolation function to find the time at which the closure temperature is reached in the thermal history
    - We will use the NumPy function `np.interp()` for this interpolation. The `np.interp()` function accepts 3 values (at least) to find the interpolated value. The values in our case would be the closure temperature, the temperature history array, and finally the time history array. You can read more about the `np.interp()` function in the [NumPy documentation online](https://numpy.org/doc/stable/reference/generated/numpy.interp.html).
        - **Note**: One challenge with the `np.interp()` function is that it requires the array of temperatures to be in increasing order, rather than decreasing as we have it now. You can use the `np.flip()` function to reverse the order of the items in the temperature history when you do the interpolation. You can find more information about the `np.flip()` function in the [NumPy documentation online](https://numpy.org/doc/stable/reference/generated/numpy.flip.html).
            - **Note**: The approach of flipping the temperature history described above allows us to use a nice "trick" here. We have a time array that is in increasing order from 0.0 Myr at the start of the thermal history to some final value (50 Myr, for example), but thermochronometer ages are normally reported as the number of millions of years ago (Ma). Normally, we would convert from our time history values (starting from 0.0) to millions of years ago by subtracting the interpolated age from the highest value in the time history, which would give a time in millions of years ago. However, in this case by interpolating with a flipped temperature array without flipping the time history, the value we get for the interpolated age will be in millions of years ago already. Cool!
- You can assign the value from the interpolation to a variable such as `age` and return that value at the end of the function definition

**What to do for this part:**

- Create a function called `calculate_age()` in your `introqg_functions.py` script file
    - Your function parameters should be `time_history`, `temp_history`, and `closure_temp` as described above

In [ ]:
# Import your calculate_age function below

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# The test below should work

# Import libraries we need
from nose.tools import assert_equal
from introqg_functions import calculate_age

# Calculate AHe age
ahe_age1 = calculate_age(
    time_history=time_hist_test1,
    temp_history=temp_hist_test1,
    closure_temp=ahe_tc1
)

# Print age
print(f"Apatite (U-Th)/He age: {ahe_age1:.1f} Ma. Expected value: 7.9 Ma.")

# Check that the AHe closure temperature is correct
assert_equal(round(ahe_age1, 3), 7.902)

In [ ]:
# The test below should work

# Calculate ZHe age
zhe_age1 = calculate_age(
    time_history=time_hist_test2,
    temp_history=temp_hist_test2,
    closure_temp=zhe_tc2
)

# Print age
print(f"Zircon (U-Th)/He age: {zhe_age1:.1f} Ma. Expected value: 38.3 Ma.")

# Check that the ZHe closure temperature is correct
assert_equal(round(zhe_age1, 3), 38.287)

In [ ]:
# The test below should work

# Calculate MAr age
mar_age1 = calculate_age(
    time_history=time_hist_test3,
    temp_history=temp_hist_test3,
    closure_temp=mar_tc3
)

# Print age
print(f"Muscovite Ar/Ar age: {mar_age1:.1f} Ma. Expected value: 20.9 Ma.")

# Check that the MAr closure temperature is correct
assert_equal(round(mar_age1, 3), 20.916)

### Part 4: Testing the model (2 points)

At this point we have all of the components we need to see how this thermochronometer age prediction model works. We simply need to combine the different parts in order to calculate some predicted thermochronometer ages. Let's do that below.

**Combining the model components**

Below is the list of items we need to do in order to test our numerical model.

1. Define some parameters used in the model. In our case, we need to define the following:
    - A total simulation time (variable `total_time`) in Myr: `50.0`
    - The number of points where the depth, temperature, and time are stored (variable `num_pts`): `401`
    - The advection velocity in the thermal model (variable `velocity`) in km/Myr: `0.8`
    - The initial thermal gradient in the thermal model (variable `initial_gradient`) in °C/km: `10.0`
    - The thermal diffusivity for the thermal model (variable `diffusivity`) in km$^{2}$/Myr: `32.0`
2. Create the time and depth histories (variables `time_history`, `depth_history`)
    - You will need to use the `np.linspace()` function for this. You can see how it can be done by looking at the tests for Part 1 of this problem.
3. Create a temperature history using your `calculate_temp_history()` function using the variables defined above and storing the temperature history as `temp_history`, for example.
4. Calculate the closure temperatures for all three thermochronometers using your `calculate_closure_temps()` function
    - Assign the calculated closure temperatures to the variables `ahe_tc`, `zhe_tc`, and `mar_tc`
5. Use your `calculate_age()` function to calculate a thermochronometer age for all three thermochronometers
    - **Note**: If a closure temperature for a given thermochronometer is `None` then the calculated age will be `nan`

That's it! At this point you will have calculated any thermochronometer ages where the thermal history passes through the closure temperature. If all goes well, we'll now be in position to explore how the model works in Problem 2.

**What to do for this part:**

- Follow the instructions above to calculate some thermochronometer ages
    - Be sure to use the variable names `ahe_tc`, `zhe_tc`, and `mar_tc` for the closure temperatures
    - Be sure to use the variable names `ahe_age`, `zhe_age`, and `mar_age` for the ages
    - The other variable names will not matter, as long as you define them so that your functions use the correct values

In [ ]:
# Fill in the cell below in order to calculate the thermochronometer ages

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Print the predicted closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc:.1f} °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc:.1f} °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc}.")
print("")
print("The expected apatite (U-Th)/He closure temperature is 83.5 °C.")


In [ ]:
# Print the predicted ages
print(f"Apatite (U-Th)/He age: {ahe_age:.1f} Ma.")
print(f"Zircon (U-Th)/He age: {zhe_age:.1f} Ma.")
print(f"Muscovite Ar/Ar age: {mar_age:.1f} Ma.")
print("")
print("The expected zircon (U-Th)/He age is 11.5 Ma.")


## Problem 2: Exploring temperatures and thermochronometer ages (6 points)

After doing the heavy lifting in Problem 1, we will now apply the numerical model we have created to see how thermochronometer ages vary for different advection velocities and initial geothermal gradients.

### Scores for this problem

**Your score on this problem will be based on following criteria**:

- Using the numerical model created in Part 4 of Problem 1 to explore different advection velocities and initial thermal gradients
- Including comments that explain what most lines in the code do
- Uploading your notebook to your GitHub repository for this week's exercise

### Part 1: Slower advection (2 points)

We can first start our exploration of how different advection velocities affect thermochronometer ages by considering a case of slower advection (compared to Part 4 in Problem 1). You can use the same model steps as in Part 4 of Problem 1 with the following parameters.

- A total simulation time (variable `total_time`) in Myr: `50.0`
- The number of points where the depth, temperature, and time are stored (variable `num_pts`): `401`
- The advection velocity in the thermal model (variable `velocity`) in km/Myr: `0.4`
- The initial thermal gradient in the thermal model (variable `initial_gradient`) in °C/km: `10.0`
- The thermal diffusivity for the thermal model (variable `diffusivity`) in km$^{2}$/Myr: `32.0`

**What to do for this part:**

- Follow the instructions above to calculate some thermochronometer ages
    - Be sure to use the variable names `ahe_tc`, `zhe_tc`, and `mar_tc` for the closure temperatures
    - Be sure to use the variable names `ahe_age`, `zhe_age`, and `mar_age` for the ages
    - The other variable names will not matter, as long as you define them so that your functions use the correct values

In [ ]:
# Fill in the cell below in order to calculate the thermochronometer ages

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Print the predicted closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc:.1f} °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc:.1f} °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc}.")
print("")
print("The expected apatite (U-Th)/He closure temperature is 74.9 °C.")


In [ ]:
# Print the predicted ages
print(f"Apatite (U-Th)/He age: {ahe_age:.1f} Ma.")
print(f"Zircon (U-Th)/He age: {zhe_age:.1f} Ma.")
print(f"Muscovite Ar/Ar age: {mar_age:.1f} Ma.")
print("")
print("The expected zircon (U-Th)/He age is 38.5 Ma.")


### Part 2: Faster advection (2 points)

Now let's consider a faster advection velocity. Again, you can use the same model steps as in Part 4 of Problem 1 with the following parameters.

- A total simulation time (variable `total_time`) in Myr: `50.0`
- The number of points where the depth, temperature, and time are stored (variable `num_pts`): `401`
- The advection velocity in the thermal model (variable `velocity`) in km/Myr: `1.6`
- The initial thermal gradient in the thermal model (variable `initial_gradient`) in °C/km: `10.0`
- The thermal diffusivity for the thermal model (variable `diffusivity`) in km$^{2}$/Myr: `32.0`

**What to do for this part:**

- Follow the instructions above to calculate some thermochronometer ages
    - Be sure to use the variable names `ahe_tc`, `zhe_tc`, and `mar_tc` for the closure temperatures
    - Be sure to use the variable names `ahe_age`, `zhe_age`, and `mar_age` for the ages
    - The other variable names will not matter, as long as you define them so that your functions use the correct values

In [ ]:
# Fill in the cell below in order to calculate the thermochronometer ages

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Print the predicted closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc:.1f} °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc:.1f} °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc:.1f} °C.")
print("")
print("The expected apatite (U-Th)/He closure temperature is 95.0 °C.")


In [ ]:
# Print the predicted ages
print(f"Apatite (U-Th)/He age: {ahe_age:.1f} Ma.")
print(f"Zircon (U-Th)/He age: {zhe_age:.1f} Ma.")
print(f"Muscovite Ar/Ar age: {mar_age:.1f} Ma.")
print("")
print("The expected muscovite Ar/Ar age is 8.1 Ma.")


### Part 3: A higher initial geothermal gradient (2 points)

Finally, let's now consider a higher initial geothermal gradient and its affect on our predicted thermochronometer ages and closure temperatures. Again, you can use the same model steps as in Part 4 of Problem 1 with the following parameters.

- A total simulation time (variable `total_time`) in Myr: `50.0`
- The number of points where the depth, temperature, and time are stored (variable `num_pts`): `401`
- The advection velocity in the thermal model (variable `velocity`) in km/Myr: `0.8`
- The initial thermal gradient in the thermal model (variable `initial_gradient`) in °C/km: `20.0`
- The thermal diffusivity for the thermal model (variable `diffusivity`) in km$^{2}$/Myr: `32.0`

**What to do for this part:**

- Follow the instructions above to calculate some thermochronometer ages
    - Be sure to use the variable names `ahe_tc`, `zhe_tc`, and `mar_tc` for the closure temperatures
    - Be sure to use the variable names `ahe_age`, `zhe_age`, and `mar_age` for the ages
    - The other variable names will not matter, as long as you define them so that your functions use the correct values

In [ ]:
# Fill in the cell below in order to calculate the thermochronometer ages

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Print the predicted closure temperatures
print(f"Apatite (U-Th)/He closure temperature: {ahe_tc:.1f} °C.")
print(f"Zircon (U-Th)/He closure temperature: {zhe_tc:.1f} °C.")
print(f"Muscovite Ar/Ar closure temperature: {mar_tc:.1f} °C.")
print("")
print("The expected apatite (U-Th)/He closure temperature is 89.2 °C.")


In [ ]:
# Print the predicted ages
print(f"Apatite (U-Th)/He age: {ahe_age:.1f} Ma.")
print(f"Zircon (U-Th)/He age: {zhe_age:.1f} Ma.")
print(f"Muscovite Ar/Ar age: {mar_age:.1f} Ma.")
print("")
print("The expected zircon (U-Th)/He age is 5.4 Ma.")


## Optional reflection questions (0 points)

After completing the coding and running the models above, consider the following questions. Please answer them briefly in the Markdown cell below.

1. Based on what you observed in the comparison of the predicted ages in Part 4 of Problem 1 and in Problem 2, how does the advection velocity affect the thermochronometer ages? Why might this be?
2. Based on what you observed in the comparison of the predicted closure temperatures in Part 4 of Problem 1 and in Problem 2, how does the advection velocity affect the thermochronometer closure temperatures? Why might this be?
3. In some cases you may have asked for a closure temperature to be calculated for the muscovite Ar/Ar thermochronometer, but received a value of `None`. What does that tell you about the relationship between the closure temperature and the temperature history from the thermal model?

YOUR ANSWER HERE